In [ ]:
import requests
import json

url = "http://localhost:11434/api/generate"

headers = {
    "Content-Type": "application/json",
}


In [ ]:

data = {
    "model": "tinyllama:chat",
    "prompt": "[INST] why is the sky blue? [/INST]",
    "raw": True,
    "stream": False
}

response = requests.post(url, json=data, headers=headers)

print("Status Code:", response.status_code)
print("Response:")
print(response.json())
# print(response.text)

# formatted_result = json.dumps(response.json(), indent=2)
# # print(formatted_result)
# print(formatted_result[response])





In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
print(messages)

In [ ]:
# LangChain supports many other chat models. Here, we're using Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters
llm = ChatOllama(model="tinyllama:chat")
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# using LangChain Expressive Language chain syntax
# learn more about the LCEL on
# https://python.langchain.com/docs/expression_language/why
chain = prompt | llm | StrOutputParser()

# for brevity, response is printed in terminal
# You can use LangServe to deploy your application for
# production
print(chain.invoke({"topic": "Space travel"}))


In [15]:
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# https://colab.research.google.com/drive/1D3i-4yiPvRmUX7PWWiat7iNX-HoggKS9?usp=sharing#scrollTo=zOfAmQ7LwpJC

# llm = Ollama(model="tinyllama:chat")
llm = Ollama(model="llama2")


from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

component_name_schema = ResponseSchema(name="component_name",
                             description="This is the name of the component or device")

reasoning_schema = ResponseSchema(name="reasoning",
                                      description="just reasoning")

component_status_schema = ResponseSchema(name="component_status",
                                    description="On or off")

response_schemas = [component_name_schema, 
                    reasoning_schema,
                    component_status_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


prompt = PromptTemplate.from_template("""

Select the tools as best as you can. You can also ignore if not possible. 

example : 
                                      
Question: Switch On the light1
component_name: light1
reasoning: we have to switch it on
component_status: on                                  

Question: Switch off the light1
component_name: light1
reasoning: we have to switch it off
component_status: off                                   

                              


Read the Question1 and answer using the following format for the output:
{format_instructions}                                                         

                                      
Begin!
        
Question1 : {input}
                             
""")

chain = prompt | llm 



response = chain.invoke({'input':"Turn down the l5" , 'format_instructions':format_instructions })

# print(data)

response_as_dict = output_parser.parse(response)
print(response_as_dict)

OutputParserException: Got invalid return object. Expected key `component_name` to be present, but got {'componenent_name': 'Lamp 5', 'reasoning': 'We have to turn it down.', 'compoben_status': 'Off'}